In [8]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import statsmodels.api as sm
from IPython.display import Image

half_hourly_data = pd.read_csv('Input/halfhourly_dataset/halfhourly_dataset/block_0.csv')


In [15]:
import pandas as pd
from datetime import datetime, timedelta

# Load your half-hourly data
half_hourly_data = pd.read_csv('Input/halfhourly_dataset/halfhourly_dataset/block_0.csv')

# Convert 'tstp' column to datetime
half_hourly_data['tstp'] = pd.to_datetime(half_hourly_data['tstp'], format='%Y-%m-%d %H:%M:%S.%f')

# Sort the data by LCLid and tstp
half_hourly_data.sort_values(by=['LCLid', 'tstp'], inplace=True)

# Group the data by LCLid
grouped = half_hourly_data.groupby('LCLid')

# Define a function to fill missing records for each group
def fill_missing(group):
    # Create a date range for 48 records (assuming the data starts at midnight)
    start_time = group['tstp'].min().replace(hour=0, minute=0, second=0, microsecond=0)
    end_time = group['tstp'].max()
    date_range = pd.date_range(start=start_time, end=end_time, freq='30T')

    # Check which records are missing
    missing_records = date_range[~date_range.isin(group['tstp'])]

    # Create empty records for missing times
    missing_data = pd.DataFrame({'tstp': missing_records, 'LCLid': group['LCLid'].iloc[0], 'energy(kWh/hh)': 0.0})

    # Concatenate the missing data with the existing data for the group
    return pd.concat([group, missing_data])

# Apply the fill_missing function to each group and concatenate the results
filled_data = grouped.apply(fill_missing)

# Reset the index of the resulting DataFrame
filled_data.reset_index(drop=True, inplace=True)

# Sort the filled data
filled_data.sort_values(by=['LCLid', 'tstp'], inplace=True)

# Save the filled data to a CSV file
filled_data.to_csv('filled_half_hourly_data.csv', index=False)
import pandas as pd

# Load the 'filled_half_hourly_data.csv' file
data = pd.read_csv('filled_half_hourly_data.csv')

# Count the number of unique rows
unique_rows = data.drop_duplicates().shape[0]

# If the CSV contains a 'tstp' column, convert it to datetime and count unique days
if 'tstp' in data.columns:
    data['tstp'] = pd.to_datetime(data['tstp'])
    unique_days = data['tstp'].dt.date.nunique()
    print(f'Unique Rows: {unique_rows}, Unique Days: {unique_days}')
else:
    print(f'Unique Rows: {unique_rows}')


# Count the number of unique days
unique_days = filled_half_hourly_data['tstp'].dt.date.nunique()

print(f'Number of unique days in the dataset: {unique_days}')

Unique Rows: 1231776, Unique Days: 819
Number of unique days in the dataset: 819


In [20]:
import pandas as pd

# Load the 'half_hourly.csv' and 'daily_dataset.csv' files
half_hourly_data = pd.read_csv('Input/hhblock_dataset/hhblock_dataset/block_0.csv')
daily_data = pd.read_csv('Input/daily_dataset/daily_dataset/block_0.csv')


# Convert 'day' column to datetime objects
half_hourly_data['day'] = pd.to_datetime(half_hourly_data['day']).dt.date
daily_data['day'] = pd.to_datetime(daily_data['day']).dt.date

# Merge the dataframes based on 'LCLid' and 'day'
merged_data = pd.merge(half_hourly_data, daily_data, on=['LCLid', 'day'], how='left')

# Reorder columns and fill missing energy-related columns with zeros
energy_cols = ['energy_median', 'energy_mean', 'energy_max', 'energy_count', 'energy_std', 'energy_sum', 'energy_min']
merged_data[energy_cols] = merged_data[energy_cols].fillna(0)

# Save the merged data to a new CSV file
merged_data.to_csv('merged_half_hourly.csv', index=False)

In [26]:
import os
import pandas as pd

# Define the folder paths
half_hourly_folder = 'Input/hhblock_dataset/hhblock_dataset/'
daily_dataset_folder = 'Input/daily_dataset/daily_dataset/'
new_half_hourly_folder = 'new_half_hourly'  # New folder to save merged files

# Create the new folder if it doesn't exist
if not os.path.exists(new_half_hourly_folder):
    os.makedirs(new_half_hourly_folder)

# Function to process and merge data for one file
def process_and_merge(file_name, block_value=None):
    # Load half-hourly and daily data
    half_hourly_data = pd.read_csv(os.path.join(half_hourly_folder, file_name))
    daily_data = pd.read_csv(os.path.join(daily_dataset_folder, file_name))

    # Convert 'day' column to datetime objects
    half_hourly_data['day'] = pd.to_datetime(half_hourly_data['day']).dt.date
    daily_data['day'] = pd.to_datetime(daily_data['day']).dt.date

    # Merge the dataframes based on 'LCLid' and 'day'
    merged_data = pd.merge(half_hourly_data, daily_data, on=['LCLid', 'day'], how='left')

    # Reorder columns and fill missing energy-related columns with zeros
    energy_cols = ['energy_median', 'energy_mean', 'energy_max', 'energy_count', 'energy_std', 'energy_sum', 'energy_min']
    merged_data[energy_cols] = merged_data[energy_cols].fillna(0)

    # If block_value is provided, add a 'block' column with the specified value
    if block_value:
        merged_data['block'] = block_value

    # Save the merged data to the 'new_half_hourly' folder
    output_file_name = os.path.join(new_half_hourly_folder, 'merged_' + file_name)
    merged_data.to_csv(output_file_name, index=False)

# Process all files in the 'half_hourly' folder
for file_name in os.listdir(half_hourly_folder):
    # Extract the block value from the filename (e.g., block_2.csv)
    block_value = file_name.split('_')[1].split('.')[0] if 'block_' in file_name else None

    process_and_merge(file_name, block_value)


In [29]:
import pandas as pd

# Read the 'information_households.csv' file
information_file = 'Input/informations_households.csv'
information_data = pd.read_csv(information_file)
new_half_hourly_folder = 'new_half_hourly'  # New folder to save merged files

# Iterate through each unique 'file' value in the 'information_households.csv' file
for block_file in information_data['file'].unique():
    # Filter rows based on the 'file' value
    block_data = information_data[information_data['file'] == block_file]

    # Read the corresponding 'merged_block_X.csv' file from the "new_half_hourly" folder
    merged_block_file = f'merged_{block_file}.csv'
    merged_block_data = pd.read_csv(os.path.join(new_half_hourly_folder, merged_block_file))

    # Merge the data based on the 'LCLid'
    merged_data = pd.merge(merged_block_data, block_data[['LCLid', 'stdorToU', 'Acorn', 'Acorn_grouped']],
                           on='LCLid', how='left')

    # Fill missing values in the 'merged_block_X.csv' file with values from 'information_households.csv'
    for column in ['stdorToU', 'Acorn', 'Acorn_grouped']:
        if f'{column}_x' in merged_data.columns and f'{column}_y' in merged_data.columns:
            merged_data[column] = merged_data[f'{column}_x'].combine_first(merged_data[f'{column}_y'])
        elif f'{column}_x' in merged_data.columns:
            merged_data[column] = merged_data[f'{column}_x']
        elif f'{column}_y' in merged_data.columns:
            merged_data[column] = merged_data[f'{column}_y']

    # Drop the intermediate columns, if they exist
    columns_to_drop = [f'{column}_x' for column in ['stdorToU', 'Acorn', 'Acorn_grouped']] + \
                      [f'{column}_y' for column in ['stdorToU', 'Acorn', 'Acorn_grouped']]
    merged_data = merged_data.drop(columns=columns_to_drop, errors='ignore')

    # Save the updated 'merged_block_X.csv' file in the "new_half_hourly" folder
    merged_data.to_csv(os.path.join(new_half_hourly_folder, merged_block_file), index=False)

In [31]:
import pandas as pd

# Load your DataFrame from the 'weather_daily.csv' file
weather_data = pd.read_csv('Input/weather_daily_darksky.csv')

# Convert the 'time' column to datetime
weather_data['time'] = pd.to_datetime(weather_data['time'])

# Extract the date in the format 'yyyy-mm-dd' and assign it to the 'day' column
weather_data['day'] = weather_data['time'].dt.strftime('%Y-%m-%d')

# Reorder the columns with 'day' as the first column
weather_data = weather_data[['day'] + [col for col in weather_data.columns if col != 'day']]

# Save the updated DataFrame back to a CSV file
weather_data.to_csv('weather_daily_with_day.csv', index=False)


In [32]:
import os
import pandas as pd

# Define the folder paths
new_half_hourly_folder = 'new_half_hourly'  # Folder with 'new_half_hourly' files
output_folder = 'new_half_hourly_with_day'  # Folder to save merged files

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the weather data
weather_data = pd.read_csv('weather_daily_with_day.csv')

# List the files in the 'new_half_hourly' folder
files = os.listdir(new_half_hourly_folder)

# Iterate through each file in the 'new_half_hourly' folder
for file in files:
    if file.endswith('.csv'):
        # Read the 'new_half_hourly' file
        half_hourly_data = pd.read_csv(os.path.join(new_half_hourly_folder, file))

        # Merge the data based on the 'day' column
        merged_data = pd.merge(half_hourly_data, weather_data, on='day', how='left')

        # Save the updated 'new_half_hourly' file to the output folder
        output_file = os.path.join(output_folder, file)
        merged_data.to_csv(output_file, index=False)


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import os

# Define your custom RNN model
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(MyRNN, self).__init()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Define a custom dataset
class CustomDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.target = target

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.target[idx]
        return x, y

# Specify the folder path containing CSV files
folder_path = 'new_half_hourly_with_day'

# Initialize your RNN model
input_size = 1  # Modify based on your data
hidden_size = 64  # Modify as needed
num_layers = 1  # Modify based on your architecture
output_size = 1  # Modify based on your problem
rnn = MyRNN(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.001)

# Iterate through each file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        # Load your dataset (assumes 'X' contains features and 'y' contains target)
        # You will need to adapt this to load your specific data
        X = np.random.rand(100, 10, 1)  # Example random data
        y = np.random.rand(100, 1)  # Example random target
        dataset = CustomDataset(X, y)

        # Initialize DataLoader
        dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

        # Training loop
        num_epochs = 10
        for epoch in range(num_epochs):
            for inputs, labels in dataloader:
                optimizer.zero_grad()
                outputs = rnn(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                print(f'File: {file_name}, Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# You can then use the trained model for prediction on each file


ModuleNotFoundError: No module named 'pandas'